# Install the necessary packages


In [3]:
#pip install openmeteo-requests
#pip install requests-cache retry-requests numpy pandas
#pip install geopy

# Acquiring the Co-ordinates of specific cities

In [2]:
from geopy.geocoders import Nominatim

def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Example usage:
location_name = "Pune"
coordinates = get_coordinates(location_name)
if coordinates:
    print(f"Coordinates of {location_name}: {coordinates}")
else:
    print("Location not found.")


Coordinates of Pune: (18.5213738, 73.8545071)


# Using Api to acquire weather information

In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 18.5213738,
	"longitude": 73.854,
	"hourly": ["rain", "visibility", "wind_speed_180m", "temperature_180m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_rain = hourly.Variables(0).ValuesAsNumpy()
hourly_visibility = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_180m = hourly.Variables(2).ValuesAsNumpy()
hourly_temperature_180m = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["rain"] = hourly_rain
hourly_data["visibility"] = hourly_visibility
hourly_data["wind_speed_180m"] = hourly_wind_speed_180m
hourly_data["temperature_180m"] = hourly_temperature_180m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 18.625°N 74.0°E
Elevation 540.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  rain  visibility  wind_speed_180m  \
0   2024-08-26 00:00:00+00:00   0.0       140.0        32.449963   
1   2024-08-26 01:00:00+00:00   0.0       180.0        33.724911   
2   2024-08-26 02:00:00+00:00   0.0        60.0        32.449959   
3   2024-08-26 03:00:00+00:00   0.0      4240.0        30.926493   
4   2024-08-26 04:00:00+00:00   0.0     15500.0        39.523014   
..                        ...   ...         ...              ...   
163 2024-09-01 19:00:00+00:00   0.0      1980.0        29.598919   
164 2024-09-01 20:00:00+00:00   0.0      2500.0        30.293522   
165 2024-09-01 21:00:00+00:00   0.0      3000.0        30.633865   
166 2024-09-01 22:00:00+00:00   0.0      2060.0        29.888672   
167 2024-09-01 23:00:00+00:00   0.0      1140.0        28.799999   

     temperature_180m  
0           22.059999  
1           21.960001  
2        

# Writing the data into a file

In [5]:
import os

averages=hourly_dataframe.mean()
print(averages)
df=pd.DataFrame([averages],columns=averages.index)

# File path
file_path = 'weather_data.csv'

# Check if the file already exists
file_exists = os.path.isfile(file_path)

df.to_csv('flight_data.csv',mode='a',header=not file_exists,index=False)


date                2024-08-29 11:30:00+00:00
rain                                      0.0
visibility                        7165.119141
wind_speed_180m                     25.219833
temperature_180m                    23.285002
dtype: object


Run this process over multiple days to acquire large amount of data

Flight Safety Criteria

To classify flight safety, the following thresholds are used for the weather conditions:

* Rain: If the rain exceeds 2.5 cm per hour, the flight is considered unsafe.
* Visibility: If visibility is less than 1.6 km (1600 meters), the flight is considered unsafe.
* Wind Speed at 180 meters: If the wind speed exceeds 46.3 km/h, the flight is considered unsafe.
* Temperature at 180 meters: If the temperature is below -10°C or above 35°C, the flight is considered unsafe.

These criteria help in assessing whether a flight is safe or unsafe based on the provided weather conditions.